In [1]:
import pandas as pd
# Expand the data to remove elipses and make more readable
pd.set_option('display.max_colwidth',250)
pd.set_option('display.max_columns',250)
pd.set_option('display.max_rows',1000)
import numpy as np
import requests
import nba_api
import pathlib as plib
import re
import time
from nba_api.stats.static import teams,players
from nba_api.stats.endpoints import playergamelog#, gamerotation #if getting 433 error, check each endpoint individually
from nba_api.stats.endpoints import commonteamroster, commonplayerinfo
from nba_api.stats.endpoints import leaguegamefinder, playbyplay


In [2]:
##########################
######### TEAMS ##########
##########################
# Get list of all teams
nba_teams = teams.get_teams()

# User input teams
team1_abv = input("Team 1 abv:")
team2_abv = input("Team 2 abv:")

#User input season
front_half_season = input("Season Yr:")
back_half_season = str(int(front_half_season)+1)
season_id = front_half_season + '-' + back_half_season[-2:]

# Get team_id for given teams
team1 = [team for team in nba_teams if team['abbreviation'] == team1_abv][0]
team1_id = team1['id']
team2 = [team for team in nba_teams if team['abbreviation'] == team2_abv][0]
team2_id = team2['id']

team_ids = [team1_id, team2_id]
team_ids_dict = pd.DataFrame({'ids' : team_ids})
#team_ids_dict.head()

df = leaguegamefinder.LeagueGameFinder([]).get_data_frames()[0]
#df.head()
#print(type(season_id))
#print(season_id)

In [9]:
# Query for games where the Celtics were playing
#teamgamefinder = leaguegamefinder.LeagueGameFinder(team for team in nba_teams if team['id'] == team_ids)
# The first DataFrame of those returned is what we want.
teamgames = pd.DataFrame([])
for i in range(len(team_ids)):
    df_no_tgi = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_ids[i],season_nullable=season_id).get_data_frames()[0]    

    df_no_tgi['t_game_num'] = df_no_tgi.sort_values(['TEAM_ID','GAME_ID'])['GAME_ID'].index + 1
    teamgames = pd.concat([teamgames, df_no_tgi], ignore_index=True)
    time.sleep(1)
#teamgames.head()

In [4]:
##########################
######### PLAYERS ########
##########################
# Get list of current players on teams
team_rosters =  pd.DataFrame([])
for i in range(len(team_ids)):
    rosters = commonteamroster.CommonTeamRoster(team_id=team_ids[i]).get_data_frames()[0]

    team_rosters = pd.concat([team_rosters, rosters], ignore_index=True)
    time.sleep(1)
#team_rosters.head()
player_ids_list = team_rosters['PLAYER_ID'].unique()

In [5]:
## Player info
player_detail = pd.DataFrame(columns=['PERSON_ID','FIRST_NAME','LAST_NAME','DISPLAY_LAST_COMMA_FIRST','TEAM_NAME','TEAM_ID'])

for i in range(len(player_ids_list)):
    pd_df = commonplayerinfo.CommonPlayerInfo(player_id=player_ids_list[i]).get_data_frames()[0][['PERSON_ID','FIRST_NAME','LAST_NAME','DISPLAY_LAST_COMMA_FIRST','TEAM_NAME','TEAM_ID']]

    player_detail = pd.concat([player_detail, pd_df], ignore_index=True)
    time.sleep(1)

In [6]:
game_data = pd.DataFrame([])

for p in range(len(player_ids_list)):
    df_no_pgi = playergamelog.PlayerGameLog(player_id=player_ids_list[i]).get_data_frames()[0]
    
    df_no_pgi['p_game_num'] = df_no_pgi.sort_values(['Player_ID','Game_ID'])['Game_ID'].index + 1
    game_data = pd.concat([game_data, df_no_pgi], ignore_index=True)
    time.sleep(1)
#game_data.head()

In [7]:
gd_pd_merge = game_data.merge(player_detail,left_on='Player_ID',right_on='PERSON_ID')
team_games_abv = teamgames[['GAME_ID','t_game_num']]
gd_pd_merge = gd_pd_merge.merge(team_games_abv,left_on='Game_ID',right_on='GAME_ID').drop_duplicates()
gd_pd_merge = gd_pd_merge[['SEASON_ID','TEAM_NAME','TEAM_ID','Player_ID','FIRST_NAME','LAST_NAME','DISPLAY_LAST_COMMA_FIRST','Game_ID','p_game_num','t_game_num','GAME_DATE','MATCHUP','WL','MIN','FGM','FGA','FG_PCT','FG3M','FG3A','FG3_PCT','FTM','FTA','FT_PCT','OREB','DREB','REB','AST','STL','BLK','TOV','PF','PTS','PLUS_MINUS']]
#gd_pd_merge.head(40)

In [8]:
team_log = teamgames.merge(gd_pd_merge,left_on='TEAM_ID',right_on='TEAM_ID')

filepath_tl = plib.Path(r'C:\Users\Danny\git_projects\outputs\sports_analysis_outputs\team_log.csv')  
filepath_player_stats = plib.Path(r'C:\Users\Danny\git_projects\outputs\sports_analysis_outputs\player_stats.csv')  

filepath_tl.parent.mkdir(parents=True, exist_ok=True)
filepath_player_stats.parent.mkdir(parents=True, exist_ok=True)

gd_pd_merge.to_csv(filepath_player_stats)
team_log.to_csv(filepath_tl) 
#player_team_log.head(1000).sort_values(['GAME_ID']).sort_values(['PLAYER_ID'])